#### **Merging and Joining**
1. **Inner Join on Chinook Database**
   - Load the `chinook.db` database.
   - Perform an inner join between the `customers` and `invoices` tables on the `CustomerId` column.
   - Find the total number of invoices for each customer.

In [20]:
import sqlite3
import pandas as pd
import numpy as np

# Connect to the Chinook database
conn = sqlite3.connect(r"data\chinook.db")

# Load tables into DataFrames
customers = pd.read_sql_query("SELECT * FROM customers", conn)
invoices = pd.read_sql_query("SELECT * FROM invoices", conn)

# Perform inner join
merged_df = customers.merge(invoices, on="CustomerId", how="inner")

# Count the number of invoices per customer
invoice_counts = merged_df.groupby("CustomerId")["InvoiceId"].count().reset_index()

# Rename columns
invoice_counts.columns = ["CustomerId", "Total_Invoices"]

print(invoice_counts.head())

# Close connection
conn.close()


   CustomerId  Total_Invoices
0           1               7
1           2               7
2           3               7
3           4               7
4           5               7


2. **Outer Join on Movie Data**
   - Load the `movie.csv` file.
   - Create two smaller DataFrames:
     - One with only `director_name` and `color`.
     - Another with `director_name` and `num_critic_for_reviews`.
   - Perform a left join and then a full outer join on `director_name`.
   - Count how many rows are in the resulting DataFrames for each join type.

In [4]:
# Load movie dataset
movies = pd.read_csv(r"data/movie.csv")

# Create two smaller DataFrames
df1 = movies[["director_name", "color"]].drop_duplicates()
df2 = movies[["director_name", "num_critic_for_reviews"]].drop_duplicates()

# Left join
left_join_df = df1.merge(df2, on="director_name", how="left")

# Full outer join
full_outer_join_df = df1.merge(df2, on="director_name", how="outer")

# Row counts
print(f"Left Join Rows: {len(left_join_df)}")
print(f"Full Outer Join Rows: {len(full_outer_join_df)}")


Left Join Rows: 5547
Full Outer Join Rows: 5547


#### **Grouping and Aggregating**
1. **Grouped Aggregations on Titanic**
   - Group passengers by `Pclass` and calculate the following:
     - Average age.
     - Total fare.
     - Count of passengers.
   - Save the results to a new DataFrame.

In [6]:
# Load Titanic dataset
titanic = pd.read_excel(r"data\titanic.xlsx")

# Group by Pclass and aggregate
grouped_titanic = titanic.groupby("Pclass").agg(
    Avg_Age=("Age", "mean"),
    Total_Fare=("Fare", "sum"),
    Passenger_Count=("PassengerId", "count")
).reset_index()

print(grouped_titanic)


   Pclass    Avg_Age  Total_Fare  Passenger_Count
0       1  38.233441  18177.4125              216
1       2  29.877630   3801.8417              184
2       3  25.140620   6714.6951              491


2. **Multi-level Grouping on Movie Data**
   - Group the movies by `color` and `director_name`.
   - Find:
     - Total `num_critic_for_reviews` for each group.
     - Average `duration` for each group.

In [7]:
# Group by color and director_name
grouped_movies = movies.groupby(["color", "director_name"]).agg(
    Total_Critic_Reviews=("num_critic_for_reviews", "sum"),
    Avg_Duration=("duration", "mean")
).reset_index()

print(grouped_movies.head())


             color     director_name  Total_Critic_Reviews  Avg_Duration
0  Black and White    Akira Kurosawa                 153.0         202.0
1  Black and White    Aleksey German                 121.0         177.0
2  Black and White      Alex Garland                 489.0         108.0
3  Black and White   Alexander Payne                 433.0         115.0
4  Black and White  Alfred Hitchcock                 434.0         119.0


3. **Nested Grouping on Flights**
   - Group flights by `Year` and `Month` and calculate:
     - Total number of flights.
     - Average arrival delay (`ArrDelay`).
     - Maximum departure delay (`DepDelay`).

In [13]:
# Load flights dataset
flights = pd.read_parquet(r"data\flights")

# convert object dtype to numeric 
flights["ArrDelay"] = pd.to_numeric(flights["ArrDelay"], errors="coerce")
flights["DepDelay"] = pd.to_numeric(flights["DepDelay"], errors="coerce")

# Group by Year and Month
grouped_flights = flights.groupby(["Year", "Month"]).agg(
    Total_Flights=("FlightDate", "count"),
    Avg_Arrival_Delay=("ArrDelay", "mean"),
    Max_Departure_Delay=("DepDelay", "max")
).reset_index()

print(grouped_flights.head())


   Year Month  Total_Flights  Avg_Arrival_Delay  Max_Departure_Delay
0  2022     1         537902           3.981085               2512.0
1  2022    10         572287           2.085886               2963.0
2  2022    11         546410           4.708015               3433.0
3  2022    12         557095          13.767535               3169.0
4  2022     2         495713           4.674906               2545.0


#### **Applying Functions**
1. **Apply a Custom Function on Titanic**
   - Write a function to classify passengers as `Child` (age < 18) or `Adult`.
   - Use `apply` to create a new column, `Age_Group`, with these values.

In [15]:
# Function to classify age group
def classify_age(age):
    return "Child" if age < 18 else "Adult"

# Apply function
titanic["Age_Group"] = titanic["Age"].apply(classify_age)

print(titanic[["Age", "Age_Group"]].head())


    Age Age_Group
0  22.0     Adult
1  38.0     Adult
2  26.0     Adult
3  35.0     Adult
4  35.0     Adult


2. **Normalize Salaries in Chinook**
   - Load the `chinook.db` database.
   - Extract the `employees` table and normalize the salaries (`ReportsTo` column) within each department using `apply`.

In [22]:
# Load the database
conn = sqlite3.connect("data/chinook.db")

# Extract the employees table
employees = pd.read_sql_query("SELECT * FROM employees", conn)

# Manually assign random salary values (for demonstration)
np.random.seed(42)  # For reproducibility
employees["Salary"] = np.random.randint(50000, 120000, size=len(employees))

# Normalize salaries within each manager (ReportsTo)
employees["Normalized_Salary"] = employees.groupby("ReportsTo")["Salary"].transform(lambda x: (x - x.mean()) / x.std())

# Display relevant columns
print(employees[["EmployeeId", "ReportsTo", "Salary", "Normalized_Salary"]])

# Close connection
conn.close()


   EmployeeId  ReportsTo  Salary  Normalized_Salary
0           1        NaN   65795                NaN
1           2        1.0   50860          -0.707107
2           3        2.0  104886           0.898222
3           4        2.0   56265          -1.077520
4           5        2.0   87194           0.179298
5           6        1.0   94131           0.707107
6           7        6.0  110263           0.707107
7           8        6.0   66023          -0.707107


3. **Custom Function on Movies**
   - Write a function that returns `Short`, `Medium`, or `Long` based on the duration of a movie:
     - `Short`: Less than 60 minutes.
     - `Medium`: Between 60 and 120 minutes.
     - `Long`: More than 120 minutes.
   - Apply this function to classify movies in the `movie.csv` dataset.

In [19]:
# Function to categorize movies by duration
def categorize_duration(duration):
    if duration < 60:
        return "Short"
    elif 60 <= duration <= 120:
        return "Medium"
    else:
        return "Long"

# Apply function
movies["Duration_Category"] = movies["duration"].apply(categorize_duration)

print(movies[["duration", "Duration_Category"]].head())


   duration Duration_Category
0     178.0              Long
1     169.0              Long
2     148.0              Long
3     164.0              Long
4       NaN              Long


#### **Using `pipe`**
1. **Pipeline on Titanic**
   - Create a pipeline to:
     - Filter passengers who survived (`Survived == 1`).
     - Fill missing `Age` values with the mean.
     - Create a new column, `Fare_Per_Age`, by dividing `Fare` by `Age`.

In [24]:
def filter_survivors(df):
    return df[df["Survived"] == 1]  # Keeps only survivors

def fill_missing_age(df):
    df = df.copy()  # Avoids modifying a slice
    df.loc[:, "Age"] = df["Age"].fillna(df["Age"].mean())
    return df  

def add_fare_per_age(df):
    df = df.copy()  # Avoids modifying a slice
    df.loc[:, "Fare_Per_Age"] = df["Fare"] / df["Age"]
    return df  

# Load Titanic dataset (assuming `titanic` is already a DataFrame)
titanic_processed = (titanic.pipe(filter_survivors)
                              .pipe(fill_missing_age)
                              .pipe(add_fare_per_age))

print(titanic_processed.head())


   PassengerId  Survived  Pclass  \
1            2         1       1   
2            3         1       3   
3            4         1       1   
8            9         1       3   
9           10         1       2   

                                                Name     Sex   Age  SibSp  \
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
8  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  female  27.0      0   
9                Nasser, Mrs. Nicholas (Adele Achem)  female  14.0      1   

   Parch            Ticket     Fare Cabin Embarked Age_Group  Fare_Per_Age  
1      0          PC 17599  71.2833   C85        C     Adult      1.875876  
2      0  STON/O2. 3101282   7.9250   NaN        S     Adult      0.304808  
3      0            113803  53.1000  C123        S     Adult      1.517143  
8      2    

2. **Pipeline on Flights**
   - Create a pipeline to:
     - Filter flights with a departure delay greater than 30 minutes.
     - Add a column `Delay_Per_Hour` by dividing the delay by the scheduled flight duration.

In [34]:
def filter_delayed_flights(df):
    """Filter flights with a departure delay greater than 30 minutes."""
    # Convert 'DepDelayMinutes' to numeric and handle errors by coercing to NaN
    df["DepDelayMinutes"] = pd.to_numeric(df["DepDelayMinutes"], errors='coerce')
    return df[df["DepDelayMinutes"] > 30].copy()  # Make a copy

def add_delay_per_hour(df):
    """Add Delay_Per_Hour column."""
    # Ensure 'AirTime' is numeric and make a copy
    df.loc[:, "AirTime"] = pd.to_numeric(df["AirTime"], errors='coerce')
    df.loc[:, "Delay_Per_Hour"] = df["DepDelayMinutes"] / df["AirTime"]
    return df.copy()  # Make a copy to avoid modifying the original DataFrame

# Apply the pipeline
flights_processed = (flights.pipe(filter_delayed_flights)
                             .pipe(add_delay_per_hour))

# Display results
print(flights_processed[["DepDelayMinutes", "AirTime", "Delay_Per_Hour"]].head())


    DepDelayMinutes AirTime Delay_Per_Hour
11             34.0    67.0       0.507463
32             48.0    36.0       1.333333
33             34.0    36.0       0.944444
37             56.0    33.0        1.69697
44             67.0    35.0       1.914286
